# ISTM 6212 Group Porject 1

## Group Member: Gaoyu Xie, Tianwei Ma, Yuxiang Fan, Matt Sipowicz

### Q1: Download data file and header. If header data is not available construct one using the data dictionary link. Unzip the file.

In [1]:
!wget https://www.fec.gov/files/bulk-downloads/2020/indiv20.zip

--2019-09-21 18:47:36--  https://www.fec.gov/files/bulk-downloads/2020/indiv20.zip
Resolving www.fec.gov (www.fec.gov)... 99.84.216.8, 99.84.216.54, 99.84.216.78, ...
Connecting to www.fec.gov (www.fec.gov)|99.84.216.8|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com/bulk-downloads/2020/indiv20.zip [following]
--2019-09-21 18:47:37--  https://cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com/bulk-downloads/2020/indiv20.zip
Resolving cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com (cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com)... 52.95.143.40
Connecting to cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com (cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com)|52.95.143.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306302197 (292M)

In [2]:
!unzip indiv20.zip

Archive:  indiv20.zip
  inflating: itcont.txt              
  inflating: by_date/itcont_2020_20010425_20190425.txt  
  inflating: by_date/itcont_2020_20190426_20190629.txt  
  inflating: by_date/itcont_2020_20190630_20190911.txt  
  inflating: by_date/itcont_2020_invalid_dates.txt  


#### With all files downloaded, we also need the header file for the future merged csv file.

In [3]:
!wget https://www.fec.gov/files/bulk-downloads/data_dictionaries/indiv_header_file.csv

--2019-09-21 19:00:35--  https://www.fec.gov/files/bulk-downloads/data_dictionaries/indiv_header_file.csv
Resolving www.fec.gov (www.fec.gov)... 13.249.44.119, 13.249.44.3, 13.249.44.50, ...
Connecting to www.fec.gov (www.fec.gov)|13.249.44.119|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com/bulk-downloads/data_dictionaries/indiv_header_file.csv [following]
--2019-09-21 19:00:35--  https://cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com/bulk-downloads/data_dictionaries/indiv_header_file.csv
Resolving cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com (cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com)... 52.95.142.20
Connecting to cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com (cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com)|52.95.142.20|:443... connected.

#### This file contains data from 2001 to 2019, but we only need data in 2019-2020, so we elimiate date from 2001/04/25 to 2018/12/31.

In [7]:
!csvformat -d "|" by_date/itcont_2020_20010425_20190425.txt > _2020_20010425_20190425.csv

In [1]:
!csvgrep -c 14 -r "$2019" _2020_20010425_20190425.csv > _2020_20190101_20190425.csv

#### Transfer .txt files into .csv files.

In [5]:
!csvformat -d "|" by_date/itcont_2020_20190426_20190629.txt > _2020_20190426_20190629.csv

CSV contains fields longer than maximum length of 131072 characters. Try raising the maximum with the field_size_limit parameter, or try setting quoting=csv.QUOTE_NONE.


In [7]:
!csvformat -d "|" by_date/itcont_2020_20190630_20190911.txt > _2020_20190630_20190911.csv

### Q2: stack data files and header file into one file. Header need to be the first row. Find the number of lines in merged file.

In [2]:
!csvstack --no-header-row indiv_header_file.csv _2020_20190101_20190425.csv _2020_20190426_20190629.csv _2020_20190630_20190911.csv > total.csv 

#### Check the sum lines number of all three files, make sure the merged one is matched with this total number.

In [3]:
!wc -l _2020_20190101_20190425.csv _2020_20190426_20190629.csv _2020_20190630_20190911.csv 

  1960171 _2020_20190101_20190425.csv
  1437006 _2020_20190426_20190629.csv
   567312 _2020_20190630_20190911.csv
  3964489 total


In [4]:
!wc -l total.csv

3964491 total.csv


In [5]:
!touch group1.csv

#### With no header option, the csvstack will put an additional default header in the stack file. So we delete the first rows that is unnecessary.

In [6]:
!sed '1d' total.csv >> group1.csv

#### Count the number of lines of the stack file

In [7]:
!wc -l group1.csv

3964490 group1.csv


#### The total number of lines has just one more than original three files' total, because the header is added.

### Q3: extract into a new file the fields: cmte_id, transaction_pgi, entity_tp, name, city, state, zip, employer, transaction_dt, transaction_amt. Generate column stats (csvstat)

#### Picked out all requested columns and paste into a new file "group2.csv".

In [8]:
!csvcut -c CMTE_ID,TRANSACTION_PGI,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,TRANSACTION_DT,TRANSACTION_AMT group1.csv > group2.csv

#### Since our file is too large, we couldn't use csvstat directely, it showed nothing. Therefore, we first use the head to get the first few rows of file, and then have a generate view of the data file using csvstat 

In [9]:
!head group2.csv | csvcut -c 1,2,3,4,5,6,7,8,9,10 | csvstat

  1. "CMTE_ID"

	Type of data:          Text
	Contains null values:  False
	Unique values:         4
	Longest value:         9 characters
	Most common values:    C00638478 (5x)
	                       C00432906 (2x)
	                       C00640870 (1x)
	                       C00428110 (1x)

  2. "TRANSACTION_PGI"

	Type of data:          Text
	Contains null values:  False
	Unique values:         1
	Longest value:         1 characters
	Most common values:    P (9x)

  3. "ENTITY_TP"

	Type of data:          Text
	Contains null values:  False
	Unique values:         2
	Longest value:         3 characters
	Most common values:    CAN (5x)
	                       IND (4x)

  4. "NAME"

	Type of data:          Text
	Contains null values:  False
	Unique values:         5
	Longest value:         19 characters
	Most common values:    JANOWICZ, PHILIP (4x)
	                       LEMUNYON, GLENN (2x)
	                       RICHARDSON, CHARDO (1x)
	                       STEVENS, RICHARD (1x)

### Q4: using output of from question 3, sort based on cand_id or cmte_id and show the result using csvlook. Only the first 15 rows need to be shown.

In [21]:
!sort group2.csv | head -n 15 | csvlook 

sort: write failed: 'standard output': Broken pipe
sort: write error
| C00000059 | P | IND | ALDERSON, MONICA F MRS.   | PARKVILLE             | MO |   641528703 | HALLMARK CARDS, INC.              |  08312019 |  37 |
| --------- | - | --- | ------------------------- | --------------------- | -- | ----------- | --------------------------------- | --------- | --- |
| C00000059 | P | IND | ALLMAN, ELIZABETH A. MRS. | KANSAS CITY           | MO | 641,161,664 | HALLMARK MARKETING COMPANY, LLC   | 8,312,019 |  37 |
| C00000059 | P | IND | ANDERSON, SALLY ANNE MRS. | VILLAGE OF LOCH LLOYD | MO | 640,124,166 | HALLMARK CARDS, INC.              | 8,312,019 |  40 |
| C00000059 | P | IND | ANGELL, TRESSA D. MRS.    | SHAWNEE               | KS | 662,262,844 | HALLMARK MARKETING COMPANY, LLC   | 8,312,019 |  37 |
| C00000059 | P | IND | ARN, DWIGHT CALVIN MR.    | OVERLAND PARK         | KS | 662,123,448 | HALLMARK CARDS, INC.              | 4,302,019 |  67 |
| C00000059 | P | IND | ARN, DWIGHT C

#### Note: We have tried "csvsort" to sort the data, but it did not give out a result of our question (we believe the reason is our file too big to generate a result). So we instead use "sort" in this questions (and we picked only first 50k lines to run the code for answering the question because the merged one is too big).

In [10]:
!csvcut group2.csv | head -n 50000 > group2_50000.csv

In [11]:
!cat group2_50000.csv | head -n 15 | csvlook #To take a look on what the file with 50000 lines looks like.

cat: write error: Broken pipe
| CMTE_ID   | TRANSACTION_PGI | ENTITY_TP | NAME                  | CITY         | STATE |    ZIP_CODE | EMPLOYER                 | TRANSACTION_DT | TRANSACTION_AMT |
| --------- | --------------- | --------- | --------------------- | ------------ | ----- | ----------- | ------------------------ | -------------- | --------------- |
| C00432906 | P               | IND       | LEMUNYON, GLENN       | WASHINGTON   | DC    |      20,002 |                          |      1,182,019 |             500 |
| C00432906 | P               | IND       | LEMUNYON, GLENN       | WASHINGTON   | DC    |      20,002 |                          |      1,182,019 |            -500 |
| C00638478 | P               | CAN       | JANOWICZ, PHILIP      | BUENA PARK   | CA    | 906,211,685 | FRIENDS OF PHIL JANOWICZ |      1,282,019 |          35,000 |
| C00640870 | P               | CAN       | RICHARDSON, CHARDO    | LONGWOOD     | FL    | 327,505,420 | INFORMATION REQUESTED    |    

In [12]:
!csvsort -c CMTE_ID group2_50000.csv | head -n 15 | csvlook 

| CMTE_ID   | TRANSACTION_PGI | ENTITY_TP | NAME                                | CITY                | STATE |    ZIP_CODE | EMPLOYER                               | TRANSACTION_DT | TRANSACTION_AMT |
| --------- | --------------- | --------- | ----------------------------------- | ------------------- | ----- | ----------- | -------------------------------------- | -------------- | --------------- |
| C00000059 | P               | IND       | HALL, DAVID HALL 208.33 5830 OVERH  | MISSION HILLS       | KS    | 662,081,146 | 66208                                  |      1,312,019 |             416 |
| C00000059 | P               | IND       | HALL, DONALD HALL 208.33 5930 OVERH | MISSION HILLS       | KS    | 662,081,211 | 66208                                  |      1,312,019 |             416 |
| C00000422 | P               | IND       | SCHLECHTER, BENJAMIN MD             | WYOMISSING          | PA    | 196,103,341 | SPRING RIDGE PLASTIC SURGERY           |      1,062,019 |         

### Q5: same as question 4 but only display rows that matched a certain candidate id or cmte id. Only the first 15 rows need to be shown.

In [13]:
!csvgrep -c1 -r "C00428110" group2.csv | head -n 15 | csvlook

| CMTE_ID   | TRANSACTION_PGI | ENTITY_TP | NAME                       | CITY    | STATE | ZIP_CODE | EMPLOYER               | TRANSACTION_DT | TRANSACTION_AMT |
| --------- | --------------- | --------- | -------------------------- | ------- | ----- | -------- | ---------------------- | -------------- | --------------- |
| C00428110 | P               | IND       | PILLA, JOHN ANTHONY        | WICHITA | KS    |   67,210 | SPIRIT AEROSYSTEMS INC |      1,102,019 |             250 |
| C00428110 | P               | IND       | GENTILE, THOMAS C          | WICHITA | KS    |   67,210 | SPIRIT AEROSYSTEMS INC |      1,102,019 |             416 |
| C00428110 | P               | IND       | HAWKINS, DUANE F           | WICHITA | KS    |   67,210 | SPIRIT AEROSYSTEMS INC |      1,102,019 |             250 |
| C00428110 | P               | IND       | KONDROTIS, KRISSTIE A      | WICHITA | KS    |   67,210 | SPIRIT AEROSYSTEMS INC |      1,102,019 |             250 |
| C00428110 | P             

### Q6: use csv format to create a tab delimited output file for question 4. Quote all string fields.

#### Create a tab delimited and quote all string fields

In [14]:
!csvformat -T -U 2 group2.csv > group3.csv 

#### Check if it transfromed into correct format

In [15]:
!cat group3.csv | head -n 10 

"CMTE_ID"	"TRANSACTION_PGI"	"ENTITY_TP"	"NAME"	"CITY"	"STATE"	"ZIP_CODE"	"EMPLOYER"	"TRANSACTION_DT"	"TRANSACTION_AMT"
"C00432906"	"P"	"IND"	"LEMUNYON, GLENN"	"WASHINGTON"	"DC"	"20002"	""	"01182019"	"500"
"C00432906"	"P"	"IND"	"LEMUNYON, GLENN"	"WASHINGTON"	"DC"	"20002"	""	"01182019"	"-500"
"C00638478"	"P"	"CAN"	"JANOWICZ, PHILIP"	"BUENA PARK"	"CA"	"906211685"	"FRIENDS OF PHIL JANOWICZ"	"01282019"	"35000"
"C00640870"	"P"	"CAN"	"RICHARDSON, CHARDO"	"LONGWOOD"	"FL"	"327505420"	"INFORMATION REQUESTED"	"01242019"	"76"
"C00638478"	"P"	"IND"	"STEVENS, RICHARD"	"DIAMOND BAR"	"CA"	"917653902"	"NOT EMPLOYED"	"01252019"	"-50"
"C00638478"	"P"	"CAN"	"JANOWICZ, PHILIP"	"BUENA PARK"	"CA"	"906211685"	"FRIENDS OF PHIL JANOWICZ"	"01282019"	"59816"
"C00638478"	"P"	"CAN"	"JANOWICZ, PHILIP"	"BUENA PARK"	"CA"	"906211685"	"FRIENDS OF PHIL JANOWICZ"	"01282019"	"50000"
"C00638478"	"P"	"CAN"	"JANOWICZ, PHILIP"	"BUENA PARK"	"CA"	"906211685"	"FRIENDS OF PHIL JANOWICZ"	"01282019"	"33000"
"C00428110"	"P"	"IND"	"PI

### Q7:find the total number of transactions for each entity type by month (for year 2019).

#### Select Entity_type, transaction_date from group1.csv. Using a loop and pipelines to generate the the total number of transcations for each Entity by month.

In [16]:
!for m in `seq 1 8`; do echo 'month' 0$m; csvcut -c ENTITY_TP,TRANSACTION_DT group1.csv | sed '1d' | csvgrep -c 2 -r "$2019" | csvgrep -c 2 -r "^0$m" | csvcut -c 1 | sort | uniq -c | head; done

month 01
      1 
    154 CAN
      4 CCM
     11 COM
 293947 IND
    238 ORG
     15 PAC
      2 PTY
month 02
      1 
    129 CAN
      4 CCM
     14 COM
 419778 IND
    411 ORG
     10 PAC
month 03
      2 
    259 CAN
     55 CCM
     14 COM
 712951 IND
    673 ORG
    159 PAC
      1 PTY
month 04
      2 
    317 CAN
      5 CCM
     17 COM
 671069 IND
    365 ORG
     20 PAC
month 05
      3 
    262 CAN
     45 CCM
     19 COM
 598123 IND
    444 ORG
     38 PAC
month 06
      5 
    412 CAN
     27 CCM
     14 COM
 841277 IND
    601 ORG
    340 PAC
      1 PTY
month 07
    123 CAN
      1 CCM
      1 COM
 379522 IND
    136 ORG
     63 PAC
      2 PTY
month 08
      4 CAN
      1 CCM
  42220 IND
      7 ORG
      2 PAC
